In [1]:
cd ../

/Users/seiji.kamiya/ms1/python-part-ja-1-cap-2


In [2]:
import pandas as pd
import numpy as np
import json

# custom module
import data_manager.base_db_manager as db_manager

In [3]:
df_car_categories = pd.read_csv("./asset/car_categories.csv")
df_car_categories["CategoryID"] += 1
display(df_car_categories)

df_car_models = pd.read_csv("./asset/models.csv").drop(columns=["Unnamed: 0"])

df_car_models["CategoryID"] = np.vectorize(lambda x: 0 if x<4 else 1 
                                           if x<9 else 2 if x<13 else 3 
                                           if x<14 else 4 if x<24 else 5
                                           if x<28 else 6 if x<34 else 7)(df_car_models["model_id"])
df_car_models["CategoryID"] += 1
df_car_models.columns = ["ModelID", "ModelName", "PriceLow", "PriceHigh", "ImageURL", "CategoryID"]
df_car_models = df_car_models[["ModelID", "ModelName", "CategoryID", "ImageURL"]]
df_car_models["ModelID"] += 1
display(df_car_models)

,CategoryID,CategoryName
0,1,コンパクト
1,2,ミニバン
2,3,セダン
3,4,ワゴン
4,5,SUV
5,6,スポーツ
6,7,GR/GR SPORT
7,8,軽自動車


,ModelID,ModelName,CategoryID,ImageURL
0,1,アクア,1,https://toyota.jp/pages/contents/carlineup/ima...
1,2,カローラ スポーツ,1,https://toyota.jp/pages/contents/carlineup/ima...
2,3,ヤリス,1,https://toyota.jp/pages/contents/carlineup/ima...
3,4,ルーミー,1,https://toyota.jp/pages/contents/carlineup/ima...
4,5,アルファード,2,https://toyota.jp/pages/contents/carlineup/ima...
5,6,ヴェルファイア,2,https://toyota.jp/pages/contents/carlineup/ima...
6,7,ヴォクシー,2,https://toyota.jp/pages/contents/carlineup/ima...
7,8,シエンタ,2,https://toyota.jp/pages/contents/carlineup/ima...
8,9,ノア,2,https://toyota.jp/pages/contents/carlineup/ima...
9,10,カローラ,3,https://toyota.jp/pages/contents/carlineup/ima...


In [4]:
df_car_grades = pd.read_csv("./asset/grades.csv").drop(columns=["Unnamed: 0"])

df_car_grades_enrich = df_car_grades.copy()
df_car_grades_enrich["BaseID"] = range(1,len(df_car_grades_enrich)+1)
df_car_grades_enrich["GradeID"] = range(1,len(df_car_grades_enrich)+1)
df_car_grades_enrich["ModelID"] = df_car_grades_enrich["model_id"] + 1
df_car_grades_enrich["GradeName"] = df_car_grades_enrich["grade_name"] 
df_car_grades_enrich["Description"] = df_car_grades_enrich.desc
df_car_grades_enrich["EngineType"] = df_car_grades_enrich.desc.apply(
    lambda x: x.split("／")[0]).apply(
    lambda x: x if x in ['ハイブリッド', 'ガソリン', 'FCV', 'PHV', 'ディーゼル', 'EVシステム'] else "ハイブリッド")
df_car_grades_enrich["EngineID"] = df_car_grades_enrich["EngineType"].map(
    {k:i+1 for i,k in enumerate(['ハイブリッド', 'ガソリン', 'FCV', 'PHV', 'ディーゼル', 'EVシステム'])})
df_car_grades_enrich["BasePrice"] = df_car_grades_enrich["price"].apply(lambda x: 5450000 if x in ("AX", "GRSPORT", "ZGRSPORT") else int(x))
df_base = df_car_grades_enrich[["BaseID", "GradeID", "EngineID", "BasePrice"]].copy()
df_engines = df_car_grades_enrich[["EngineID", "EngineType"]].drop_duplicates().sort_values(by="EngineID")
df_car_grades = df_car_grades_enrich[["GradeID", "GradeName", "Description", "ModelID"]].copy()

df_base["Rank"] = np.random.choice(range(len(df_car_grades)),size=len(df_car_grades))

def temp_scaler(x):
    return (x-x.min())/(x.max()-x.min())

# コスト計算のためのパラメータ
df_base["FuelEfficiency"] = (
    1 + temp_scaler(df_base["BasePrice"].astype(int))
) * 10  # 10~20km/Lくらいに収める
df_base["FuelCostPerKilo"] = 160 / df_base["FuelEfficiency"]  # リッター160円で計算
df_base["MonthlyMainteCost"] = (
    df_base["BasePrice"] * 0.0005
)  # 月額のメンテコスト。毎月千円くらい
df_base["MonthlyInsuranceCost"] = (
    df_base["BasePrice"] * 0.005
)  # 月額の保険コスト。毎月1万円くらい
df_base["MonthlyParkingCost"] = (
    df_base["BasePrice"] * 0.005
)  # 月額の駐車場コスト。毎月1万円くらい
df_base["MonthlyPriceDropRate"] = (
    1 - temp_scaler(df_base["BasePrice"])
) * 0.02 + 0.01  # 月額の価格下落率。1~3%、年間10~35%程度を想定
display(df_base)
display(df_engines)
display(df_car_grades)

,BaseID,GradeID,EngineID,BasePrice,Rank,FuelEfficiency,FuelCostPerKilo,MonthlyMainteCost,MonthlyInsuranceCost,MonthlyParkingCost,MonthlyPriceDropRate
0,1,1,1,2659000,32,12.231817,13.080640,26590.0,26590.0,26590.0,0.025536
1,2,2,1,2294000,0,11.778952,13.583551,22940.0,22940.0,22940.0,0.026442
2,3,3,1,2667000,23,12.241743,13.070034,26670.0,26670.0,26670.0,0.025517
3,4,4,1,2146000,25,11.595325,13.798665,21460.0,21460.0,21460.0,0.026809
4,5,5,1,2763000,191,12.360853,12.944091,27630.0,27630.0,27630.0,0.025278
...,...,...,...,...,...,...,...,...,...,...,...
263,264,264,2,959200,235,10.122832,15.805854,9592.0,9592.0,9592.0,0.029754
264,265,265,2,1091200,154,10.286608,15.554205,10912.0,10912.0,10912.0,0.029427
265,266,266,2,1232000,256,10.461302,15.294464,12320.0,12320.0,12320.0,0.029077
266,267,267,2,1058200,170,10.245664,15.616363,10582.0,10582.0,10582.0,0.029509


,EngineID,EngineType
0,1,ハイブリッド
9,2,ガソリン
121,3,FCV
126,4,PHV
164,5,ディーゼル
181,6,EVシステム


,GradeID,GradeName,Description,ModelID
0,1,GR SPORT,ハイブリッド／1.5L／CVT／2WD FF／定員 5名,1
1,2,G,ハイブリッド／1.5L／CVT／2WD FF／定員 5名,1
2,3,Z Raffine,ハイブリッド／1.5L／CVT／2WD FF／定員 5名,1
3,4,X,ハイブリッド／1.5L／CVT／2WD FF／定員 5名,1
4,5,Z,ハイブリッド／1.5L／CVT／E-Four／定員 5名,1
...,...,...,...,...
263,264,L SA III,ガソリン／0.7L／CVT／2WD FF／定員 4名,35
264,265,L SA III,ガソリン／0.7L／CVT／4WD／定員 4名,35
265,266,G SA III,ガソリン／0.7L／CVT／2WD FF／定員 4名,35
266,267,B SA III,ガソリン／0.7L／CVT／4WD／定員 4名,35


In [5]:
df_mapping = df_car_grades_enrich[["model_id", "grade_id", "GradeID"]]
df_mapping

,model_id,grade_id,GradeID
0,0,0,1
1,0,1,2
2,0,2,3
3,0,3,4
4,0,4,5
...,...,...,...
263,34,7,264
264,34,8,265
265,34,9,266
266,34,10,267


In [6]:
df_exteriors = pd.read_csv("./asset/exterior_parts.csv").drop(columns=["Unnamed: 0"])

df_exteriors_enrich = df_exteriors.copy()
df_exteriors_enrich = df_exteriors_enrich.merge(right=df_mapping, how="left", on=["model_id", "grade_id"])
df_exteriors_enrich = df_exteriors_enrich.rename(
    columns={"model_id":"ModelID", "name":"Item", "img_url":"ImageURL", "price":"AdditionalCost"})
df_exteriors_enrich["ExteriorID"] = range(1,len(df_exteriors_enrich)+1)
# df_exteriors_enrich["GradeExteriorID"] = range(1,len(df_exteriors_enrich)+1)
df_exteriors_enrich["GradeExteriorID"] = df_exteriors_enrich["GradeID"]
df_exteriors = df_exteriors_enrich[["ExteriorID", "Item", "ImageURL", "AdditionalCost"]]
df_grade_exteriors = df_exteriors_enrich[["GradeExteriorID", "GradeID", "ExteriorID"]]
df_grade_exteriors["GradeID"] += 1

display(df_exteriors)
display(df_grade_exteriors)

/var/folders/ks/2v1c_njx6fsftjmy3npkzrj40000gq/T/ipykernel_35230/830330105.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grade_exteriors["GradeID"] += 1


,ExteriorID,Item,ImageURL,AdditionalCost
0,1,Bi-Beam LEDヘッドランプ（マニュアルレベリング機能付）＋LEDターンランプ＋LED...,https://toyota.jp/tcv/resources/aqua/002_p_006...,110000
1,2,バイカラーLEDフォグランプ（切り替え式） LEDリヤフォグランプ付車、トヨタチームメイト無...,https://toyota.jp/tcv/resources/aqua/002_p_006...,57090
2,3,バイカラーLEDフォグランプ（切り替え式） LEDリヤフォグランプ付車、パノラミックビューモ...,https://toyota.jp/tcv/resources/aqua/002_p_006...,57090
3,4,バイカラーLEDフォグランプ（切り替え式） LEDリヤフォグランプ無車、トヨタチームメイト無...,https://toyota.jp/tcv/resources/aqua/002_p_006...,56320
4,5,バイカラーLEDフォグランプ（切り替え式） LEDリヤフォグランプ無車、パノラミックビューモ...,https://toyota.jp/tcv/resources/aqua/002_p_006...,56320
...,...,...,...,...
10687,10688,ナンバーフレーム（メッキ） フロント,https://toyota.jp/tcv/resources/pixisepoch/002...,2200
10688,10689,ナンバーフレーム（メッキ） リヤ,https://toyota.jp/tcv/resources/pixisepoch/002...,2200
10689,10690,プレミアムナンバーフレーム（ダークブラックメッキ調） フロント・リアセット,https://toyota.jp/tcv/resources/pixisepoch/002...,7700
10690,10691,プレミアムナンバーフレーム（ダークブラックメッキ調） フロント,https://toyota.jp/tcv/resources/pixisepoch/002...,3850


,GradeExteriorID,GradeID,ExteriorID
0,1,2,1
1,1,2,2
2,1,2,3
3,1,2,4
4,1,2,5
...,...,...,...
10687,268,269,10688
10688,268,269,10689
10689,268,269,10690
10690,268,269,10691


In [7]:
df_interiors = pd.DataFrame({
    "InteriorID":[1, 2, 3, 4, 5, 6],
    "Item":["デジタルインナーミラー", "フロアマット（ラグジュアリータイプ）", "ラゲージトレイ", "スマートクッショントート", "アームレスト（スライドタイプ）", "革巻きステアリング"],
    "ImageURL":["https://toyota.jp/tcv/resources/prius/005_p_002/images/in/12d.jpg", 
                "https://toyota.jp/tcv/resources/prius/005_p_002/images/in/a0as.jpg",
                "https://toyota.jp/tcv/resources/prius/005_p_002/images/in/i8b0.jpg",
                "https://toyota.jp/tcv/resources/prius/005_p_002/images/in/wed1.jpg",
                "https://toyota.jp/tcv/resources/prius/005_p_002/images/in/e7m6.jpg",
                "https://toyota.jp/tcv/resources/yaris/001_p_007/images/in/b5s3.jpg"],
    "AdditionalCost":[80000,30000,11000,10000,30000,30000],
})
display(df_interiors)

df_grade_interiors = pd.merge(
    right=df_car_grades[["GradeID"]].assign(key=1),
    left=df_interiors[["InteriorID"]].assign(key=1)
).drop(columns="key").assign(
    GradeInteriorID = lambda df: range(1,len(df)+1)
)[["GradeInteriorID", "InteriorID", "GradeID"]]
display(df_grade_interiors)

df_colors = pd.DataFrame({
    "ColorID":[1, 2, 3, 4, 5, 6],
    "ColorName":["エモーショナルレッド", "プラチナホワイトパールマイカ", "ダークグレー×クリアベージュメタリック", "ダークグレー×プラスゴールドメタリック", "ブラック×プレシャスホワイトパール","ブラック×プレシャスブロンズ"],
    "ImageURL":["https://toyota.jp/tcv/resources/prius/005_p_002/images/color/bc_3u5.jpg", 
                "https://toyota.jp/tcv/resources/aqua/002_p_006/images/color/bc_089.jpg",
                "https://toyota.jp/tcv/resources/aqua/002_p_006/images/color/bc_2wt.jpg",
                "https://toyota.jp/tcv/resources/aqua/002_p_006/images/color/bc_2ys.jpg",
                "https://toyota.jp/tcv/resources/crowncrossover/001_p_002/images/color/bc_2xw.jpg",
                "https://toyota.jp/tcv/resources/crowncrossover/001_p_002/images/color/bc_2xz.jpg"
                ],
    "AdditionalCost":[50000,50000,50000,50000,50000,50000],
})
display(df_colors)

,InteriorID,Item,ImageURL,AdditionalCost
0,1,デジタルインナーミラー,https://toyota.jp/tcv/resources/prius/005_p_00...,80000
1,2,フロアマット（ラグジュアリータイプ）,https://toyota.jp/tcv/resources/prius/005_p_00...,30000
2,3,ラゲージトレイ,https://toyota.jp/tcv/resources/prius/005_p_00...,11000
3,4,スマートクッショントート,https://toyota.jp/tcv/resources/prius/005_p_00...,10000
4,5,アームレスト（スライドタイプ）,https://toyota.jp/tcv/resources/prius/005_p_00...,30000
5,6,革巻きステアリング,https://toyota.jp/tcv/resources/yaris/001_p_00...,30000


,GradeInteriorID,InteriorID,GradeID
0,1,1,1
1,2,1,2
2,3,1,3
3,4,1,4
4,5,1,5
...,...,...,...
1603,1604,6,264
1604,1605,6,265
1605,1606,6,266
1606,1607,6,267


,ColorID,ColorName,ImageURL,AdditionalCost
0,1,エモーショナルレッド,https://toyota.jp/tcv/resources/prius/005_p_00...,50000
1,2,プラチナホワイトパールマイカ,https://toyota.jp/tcv/resources/aqua/002_p_006...,50000
2,3,ダークグレー×クリアベージュメタリック,https://toyota.jp/tcv/resources/aqua/002_p_006...,50000
3,4,ダークグレー×プラスゴールドメタリック,https://toyota.jp/tcv/resources/aqua/002_p_006...,50000
4,5,ブラック×プレシャスホワイトパール,https://toyota.jp/tcv/resources/crowncrossover...,50000
5,6,ブラック×プレシャスブロンズ,https://toyota.jp/tcv/resources/crowncrossover...,50000


In [8]:
def datafram_to_json(df):
    data = []
    for i in range(len(df)):
        data.append({c:df[c].tolist()[i] for c in df.columns})
    return data

In [9]:
db_json = {}
db_json["CarCategories"] = datafram_to_json(df_car_categories.drop(columns="CategoryID"))
db_json["CarModels"] = datafram_to_json(df_car_models.drop(columns="ModelID"))
db_json["CarGrades"] = datafram_to_json(df_car_grades.drop(columns="GradeID"))
db_json["Engines"] = datafram_to_json(df_engines.drop(columns="EngineID"))
db_json["Bases"] = datafram_to_json(df_base.drop(columns="BaseID"))
db_json["Colors"] = datafram_to_json(df_colors.drop(columns="ColorID"))
db_json["Exteriors"] = datafram_to_json(df_exteriors.drop(columns="ExteriorID"))
db_json["GradeExteriors"] = datafram_to_json(df_grade_exteriors.drop(columns="GradeExteriorID"))
db_json["Interiors"] = datafram_to_json(df_interiors.drop(columns="InteriorID"))
db_json["GradeInteriors"] = datafram_to_json(df_grade_interiors.drop(columns="GradeInteriorID"))

In [10]:
with open('domain_context/db_sample.json', 'w') as f:
    json.dump(db_json, f, indent=2)